In [1]:
SMOKE_TEST = True
# SMOKE_TEST = False

# Number of models to train - None means all models (ignored in smoke test mode)
NUM_MODELS = None

SKIP_TRAINED = True

# Path to YAML config file containing list of model configs to train
CONFIG_LIST = 'notebooks/configs/mmocr_det_model_list.yml'

# Text Detection Training


In [2]:
import warnings

# Ignore all UserWarnings emitted from any submodule of torch
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module=r"torch.*"
)
# Ignore all UserWarnings emitted from any submodule of torch
warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module=r"mmcv.*"
)

In [3]:
import yaml
from pathlib import Path

# Load model configs from YAML file
with open(CONFIG_LIST, 'r') as f:
    config_paths = yaml.safe_load(f)

# Filter out commented lines and empty entries
active_configs = [cfg for cfg in config_paths if cfg and not cfg.strip().startswith('#')]

# Map model types to checkpoint URLs
CHECKPOINT_URLS = {
    "dbnetpp_custom": "https://download.openmmlab.com/mmocr/textdet/dbnetpp/dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015/dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015_20221101_124139-4ecb39ac.pth",
    "dbnet_custom": "https://download.openmmlab.com/mmocr/textdet/dbnet/dbnet_resnet50-oclip_1200e_icdar2015/dbnet_resnet50-oclip_1200e_icdar2015_20221102_115917-bde8c87a.pth",
    "psenet_custom": "https://download.openmmlab.com/mmocr/textdet/psenet/psenet_resnet50-oclip_fpnf_600e_icdar2015/psenet_resnet50-oclip_fpnf_600e_icdar2015_20221101_131357-2bdca389.pth",
    "panet_custom": "https://download.openmmlab.com/mmocr/textdet/panet/panet_resnet18_fpem-ffm_600e_icdar2015/panet_resnet18_fpem-ffm_600e_icdar2015_20220826_144817-be2acdb4.pth",
    "textsnake_custom": "https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500/textsnake_resnet50-oclip_fpn-unet_1200e_ctw1500_20221101_134814-a216e5b2.pth",
    "fcenet_custom": "https://download.openmmlab.com/mmocr/textdet/fcenet/fcenet_resnet50-oclip_fpn_1500e_icdar2015/fcenet_resnet50-oclip_fpn_1500e_icdar2015_20221101_150145-5a6fc412.pth"
}

# Create mapping from config paths to checkpoint URLs
CONFIG_TO_CKPT = {}
for config_path in active_configs:
    config_path_obj = Path(config_path)
    # Extract model type from parent directory name
    model_type = config_path_obj.parent.name
    if model_type in CHECKPOINT_URLS:
        CONFIG_TO_CKPT[config_path] = CHECKPOINT_URLS[model_type]

print(f"Loaded {len(active_configs)} active configs:")
for config in active_configs:
    print(f"  - {config}")

ROOT_CONFIG_FOLDER = 'configs/textdet'

Loaded 6 active configs:
  - configs/textdet/textsnake_custom/textsnake_cegdr.py
  - configs/textdet/psenet_custom/psenet_cegdr.py
  - configs/textdet/panet_custom/panet_cegdr.py
  - configs/textdet/dbnetpp_custom/dbnetpp_cegdr.py
  - configs/textdet/fcenet_custom/fcenet_cegdr.py
  - configs/textdet/dbnet_custom/dbnet_cegdr.py


In [ ]:
#@title Train single model

from pathlib import Path
from mmengine.runner import Runner
import time
from mmengine import Config
from dotenv import load_dotenv
import os

if SMOKE_TEST:
    load_dotenv() # NOTE: make sure to reload notebook when changing .env to use new env variables
    os.chdir(os.path.expanduser('~/bonting-identification'))

    if not active_configs:
        raise ValueError("No active configs found in CONFIG_LIST")
    
    # Use the first active config for smoke test
    model_config = active_configs[0]
    
    cfg = Config.fromfile(model_config)
    cfg['load_from'] = CONFIG_TO_CKPT[model_config]
    cfg.visualizer.name = f'{time.localtime()}'

    # Optionally, smoke test on 1 epoch
    cfg.train_cfg['max_epochs'] = 1

    runner = Runner.from_cfg(cfg)
    result = runner.train()

/home/bonting/bonting-identification
07/18 11:00:26 - mmengine - INFO - Using env variable `MLFLOW_TRACKING_URI` with value of http://localhost:5000 to replace item in config.
07/18 11:00:26 - mmengine - INFO - Using env variable `MLFLOW_TRACKING_URI` with value of http://localhost:5000 to replace item in config.


/home/bonting/micromamba/envs/bonting-id/lib/python3.11/site-packages/mmengine/utils/package_utils.py:17: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/home/bonting/micromamba/envs/bonting-id/lib/python3.11/importlib/__init__.py:169: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  _bootstrap._exec(spec, module)


07/18 11:00:27 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.11.13 | packaged by conda-forge | (main, Jun  4 2025, 14:48:23) [GCC 13.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1497910677
    GPU 0: NVIDIA GeForce RTX 3090
    CUDA_HOME: /opt/cuda
    NVCC: Cuda compilation tools, release 12.9, V12.9.86
    GCC: gcc (GCC) 15.1.1 20250425
    PyTorch: 2.1.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_50,code

/home/bonting/micromamba/envs/bonting-id/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'data/CEGD-R_train_test'. Exception: 
  return _dataset_source_registry.resolve(
/home/bonting/micromamba/envs/bonting-id/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
ERROR:tornado.general:SEND Error: Host unreachable


In [ ]:
# !rm -rf work_dirs/*


In [ ]:
#@title Train all models

import os
from mmengine.runner import Runner
import time
from mmengine import Config
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv

if not SMOKE_TEST:
    load_dotenv() # NOTE: make sure to reload notebook when changing .env to use new env variables
    # os.chdir(os.path.expanduser('~/bonting-identification'))

    results = []
    model_configs = []
    ckpts = []

    # Determine how many models to train
    if NUM_MODELS is None:
        # Use all models when NUM_MODELS is None
        models_to_train = len(active_configs)
    else:
        # Use specified number of models
        models_to_train = min(NUM_MODELS, len(active_configs))
    
    for model_config in active_configs[:models_to_train]:
        cfg = Config.fromfile(model_config)
        cfg['load_from'] = CONFIG_TO_CKPT[model_config]
        cfg.visualizer.name = f'{time.localtime()}'

        # cfg.train_cfg['max_epochs'] = 1

        runner = Runner.from_cfg(cfg)
        result = runner.train()

        results.append(result)
        model_configs.append(Path(model_config).name.rstrip('.py'))
        ckpts.append(Path(CONFIG_TO_CKPT[model_config]).parts[-2])


In [ ]:
# results_df = pd.DataFrame(results)
# results_df.insert(0, 'model_config', model_configs)
# results_df.insert(1, 'ckpt', ckpts)
# results_df = results_df.set_index(['model_config', 'ckpt'])
# results_df.sort_values('cegdr/hmean', ascending=False, inplace=True)
# results_df


In [ ]:
# save_path = Path('reports/eval/cegdr/textdet/mmocr_finetuned_det_results.csv')
# save_path.parent.mkdir(parents=True, exist_ok=True)
# print(f'Saving results to:\n{save_path}')
# results_df.to_csv(save_path, index=True, header=True)
